# Model with residual connections. 
## Optimization of the model architecture using keras tuner with the Bayesian algorithm.

In [1]:
pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.1 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import pandas as pd
import keras_tuner

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Loading train, validation and test sets

In [4]:
ds = tfds.load('malaria', 
               split=('train[:60%]', 'train[60%:80%]', 'train[80%:]'), 
               shuffle_files=False,
              data_dir='/content/gdrive/MyDrive/datasets/Malaria',
               batch_size=32,
              download=True,
              as_supervised=True,
              with_info=False)

# Model building

In [5]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        min_delta=0,
        patience=5,
        verbose=1,
        mode="max",
        baseline=None,
        restore_best_weights=True)
    ]

In [6]:
def build_model(hp):
    '''I keep the number of filters the same in all layers in this version'''
    init_filters = hp.Choice('init_filters', [16, 32, 64])
    kernel_size = hp.Choice('kernel_size', [2, 3])
    num_resid = hp.Int('n_resid_blocks', min_value=1, max_value=6, step=1)
    inputs = keras.Input(shape=(None, None, 3))
    x = keras.layers.Rescaling(scale=1.0 / 255)(inputs)
    x = keras.layers.Conv2D(
      filters=init_filters,
      kernel_size=kernel_size,
      strides=(1, 1),
      padding="same",
      activation='relu')(x)
    x = keras.layers.MaxPooling2D(
        pool_size=(2, 2), 
        strides=None, 
        padding="same")(x)
    prev_block_output = x
    for i in range(num_resid):
        # i is the number of blocks with residual connections
      x = keras.layers.Conv2D(
          filters=init_filters,
          kernel_size=kernel_size,
          strides=(1, 1),
          padding="same",
          activation='relu')(x)
      x = keras.layers.Conv2D(
          filters=init_filters,
          kernel_size=kernel_size,
          strides=(1, 1),
          padding="same",
          activation='relu')(x)
      x = keras.layers.add([x, prev_block_output])
      prev_block_output = x
    x = keras.layers.Conv2D(
        filters=init_filters,
        kernel_size=kernel_size,
        strides=(1, 1),
        padding="same",
        activation='relu')(prev_block_output)
    x = keras.layers.GlobalMaxPooling2D()(x)
    x = keras.layers.Dense(units=init_filters,
                            activation="relu",
                          kernel_regularizer=None)(x)
    x = keras.layers.Dropout(0.2)(x)
    outputs = keras.layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='Adam',
         loss='binary_crossentropy',
         metrics=['accuracy'])    
    return model

In [7]:
tuner = keras_tuner.BayesianOptimization(
    hypermodel=build_model,
    objective='val_accuracy',
    max_trials=25,
    num_initial_points=2,
    alpha=0.0001,
    beta=2.6,
    seed=8)

In [ ]:
tuner.search(ds[0], epochs=30, validation_data=ds[1], callbacks=callbacks)

Trial 8 Complete [00h 21m 50s]
val_accuracy: 0.9649791121482849

Best val_accuracy So Far: 0.9658864140510559
Total elapsed time: 03h 49m 53s

Search: Running Trial #9

Value             |Best Value So Far |Hyperparameter
16                |64                |init_filters
3                 |3                 |kernel_size
6                 |5                 |n_resid_blocks

Epoch 1/30
517/517 [==============================] - 110s 209ms/step - loss: 0.6027 - accuracy: 0.6276 - val_loss: 0.1487 - val_accuracy: 0.9508
Epoch 2/30
517/517 [==============================] - 38s 74ms/step - loss: 0.1619 - accuracy: 0.9515 - val_loss: 0.1536 - val_accuracy: 0.9488
Epoch 3/30
517/517 [==============================] - 39s 75ms/step - loss: 0.1514 - accuracy: 0.9545 - val_loss: 0.1656 - val_accuracy: 0.9412
Epoch 4/30
517/517 [==============================] - 39s 75ms/step - loss: 0.1439 - accuracy: 0.9561 - val_loss: 0.1427 - val_accuracy: 0.9514
Epoch 5/30
517/517 [=========================

In [1]:
tuner.results_summary()

NameError: ignored

# Conclusion
Process terminated due to the Colab Usage limits.
Larger kernels and numbers of residual blocks seem to work better, so I will try to increase them in the next iteration.
The maximum number of filter also worked best, though the difference between 64 filters (0.9659) and 16 filters (0.9655 achieved before termination, might be better) is not big.